<a href="https://colab.research.google.com/github/RumoisteinKartenspiel/Bioinformatic_course_2022_project/blob/main/Selective_RFdiff_with_Backup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**RFdiffusion with Google Drive integration**
RFdiffusion is a method for structure generation, with or without conditional information (a motif, target etc). It can perform a whole range of protein design challenges as we have outlined in the RFdiffusion [manuscript](https://www.biorxiv.org/content/10.1101/2022.12.09.519842v2).

**<font color="red">NOTE:</font>** This notebook is in development, we are still working on adding all the options from the manuscript above.

For **instructions**, see end of Notebook.

See [diffusion_foldcond](https://colab.research.google.com/github/sokrypton/ColabDesign/blob/v1.1.1/rf/examples/diffusion_foldcond.ipynb) for fold conditioning functionality.

See [original version](https://colab.research.google.com/github/sokrypton/ColabDesign/blob/v1.1.1/rf/examples/diffusion_ori.ipynb) of this notebook (from 31Mar2023).

***

#Changes to original Colab
Included  possibility to mount Google drive to backup the data during the run.
- If run aborts after the RF diffusion step, just run all cells again **exept** "Pdb file upload" and "run RFdiffusion to generate a backbone"
- The project name will be create a new folder in the drive with this name, please DO NOT leave spaces, if wished use underscore "_".
- Setting the variable "name" in "run RFdiffusion to generate a backbone" will create a subfolder contianing all the information and data for this specific run, this allows also to design multiple proteins within one project.
- For rerunning, **please note the "name" of the run you want to recreate**, this will be asked for input in "Display 3D structure" and "Running ProteinMPNN" interactivly. (Please look for it at the end of the cell, where the output is printed.)
- Dont use spaces in project and run naming, will be repalced with "_"

In [ ]:
#@title setup dependencies and **RFdiffusion** (~2m)
#@markdown Please also run this when restarting a run and skipping RF diffusion step, important parameter are defined and installed here.
%%time
import os, time, signal
import sys, random, string, re
if not os.path.isdir("params"):
  os.system("apt-get install aria2")
  os.system("mkdir params")
  # send param download into background
  os.system("(\
  aria2c -q -x 16 https://files.ipd.uw.edu/krypton/schedules.zip; \
  aria2c -q -x 16 http://files.ipd.uw.edu/pub/RFdiffusion/6f5902ac237024bdd0c176cb93063dc4/Base_ckpt.pt; \
  aria2c -q -x 16 http://files.ipd.uw.edu/pub/RFdiffusion/e29311f6f1bf1af907f9ef9f44b8328b/Complex_base_ckpt.pt; \
  aria2c -q -x 16 http://files.ipd.uw.edu/pub/RFdiffusion/f572d396fae9206628714fb2ce00f72e/Complex_beta_ckpt.pt; \
  aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2022-12-06.tar; \
  tar -xf alphafold_params_2022-12-06.tar -C params; \
  touch params/done.txt) &")

if not os.path.isdir("RFdiffusion"):
  print("installing RFdiffusion...")
  os.system("git clone https://github.com/sokrypton/RFdiffusion.git")
  os.system("pip install jedi omegaconf hydra-core icecream pyrsistent pynvml decorator")
  os.system("pip install git+https://github.com/NVIDIA/dllogger#egg=dllogger")
  # 17Mar2024: adding --no-dependencies to avoid installing nvidia-cuda-* dependencies
  os.system("pip install --no-dependencies dgl==2.0.0 -f https://data.dgl.ai/wheels/cu121/repo.html")
  os.system("pip install --no-dependencies e3nn==0.3.3 opt_einsum_fx")
  os.system("cd RFdiffusion/env/SE3Transformer; pip install .")
  #os.system("wget -qnc https://files.ipd.uw.edu/krypton/ananas")
  #os.system("chmod +x ananas")

if not os.path.isdir("colabdesign"):
  print("installing ColabDesign...")
  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git@v1.1.1")
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/colabdesign colabdesign")

if not os.path.isdir("RFdiffusion/models"):
  print("downloading RFdiffusion params...")
  os.system("mkdir RFdiffusion/models")
  models = ["Base_ckpt.pt","Complex_base_ckpt.pt"]
  for m in models:
    while os.path.isfile(f"{m}.aria2"):
      time.sleep(5)
  os.system(f"mv {' '.join(models)} RFdiffusion/models")
  os.system("unzip schedules.zip; rm schedules.zip")

if 'RFdiffusion' not in sys.path:
  os.environ["DGLBACKEND"] = "pytorch"
  sys.path.append('RFdiffusion')

from google.colab import files
import json
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import ipywidgets as widgets
import py3Dmol
import os
import shutil
try:
    import Bio
except ImportError:
    !pip install biopython
    import Bio
import Bio.PDB
from Bio import PDB
import subprocess


from inference.utils import parse_pdb
from colabdesign.rf.utils import get_ca
from colabdesign.rf.utils import fix_contigs, fix_partial_contigs, fix_pdb, sym_it
from colabdesign.shared.protein import pdb_to_string
from colabdesign.shared.plot import plot_pseudo_3D

def get_pdb(pdb_code=None):

  directory_path = path + "/input_pdbs"
    # Check if the directory already exists
  if not os.path.exists(directory_path):
    # If it doesn't exist, create the directory
    os.makedirs(directory_path)

    # Specify the directory path
    print()
    if pdb_code is None or pdb_code == "" or pdb_code == "upload":
        # Use the file upload widget to upload PDB files
        uploaded = files.upload()

        print("Double check your input please, especially if the selected residue number are matching to your target residues!")
        # Load the pdb file
        parser = Bio.PDB.PDBParser()
        structure = parser.get_structure("input_pdb", list(uploaded.keys())[0])

        # Iterate over the structure
        for model in structure:
            for chain in model:
                # Get the chain ID
                chain_id = chain.id

                # Initialize variables for the current chain
                chain_length = 0
                residues = []

                # Iterate over the residues in the chain
                for residue in chain:
                    # Increment the chain length
                    chain_length += 1

                    # Add residue information to the list
                    residue_name = residue.resname
                    residue_number = residue.id[1]
                    residues.append(f"{residue_name}{residue_number}")

                # Print the chain length and all residues
                print("Here are some information about your input file:")
                print(f"Chain {chain_id}:")
                print(f"\tLength: {chain_length}")
                print(f"\tResidues: {' '.join(residues)}")

        # Move the uploaded files to the input_pdbs directory
        for filename in uploaded.keys():
            source_path = filename
            destination_path = os.path.join(directory_path, filename)
            shutil.copy2(source_path, destination_path)

        return destination_path

    elif os.path.isfile(os.path.join(directory_path, pdb_code)):
        return os.path.join(directory_path, pdb_code)


def run_ananas(pdb_str, path, sym=None):
  pdb_filename = f"{path}/ananas_input.pdb"
  out_filename = f"{path}/ananas.json"
  with open(pdb_filename,"w") as handle:
    handle.write(pdb_str)

  cmd = f"./ananas {pdb_filename} -u -j {out_filename}"
  if sym is None: os.system(cmd)
  else: os.system(f"{cmd} {sym}")

  # parse results
  try:
    out = json.loads(open(out_filename,"r").read())
    results,AU = out[0], out[-1]["AU"]
    group = AU["group"]
    chains = AU["chain names"]
    rmsd = results["Average_RMSD"]
    print(f"AnAnaS detected {group} symmetry at RMSD:{rmsd:.3}")

    C = np.array(results['transforms'][0]['CENTER'])
    A = [np.array(t["AXIS"]) for t in results['transforms']]

    # apply symmetry and filter to the asymmetric unit
    new_lines = []
    for line in pdb_str.split("\n"):
      if line.startswith("ATOM"):
        chain = line[21:22]
        if chain in chains:
          x = np.array([float(line[i:(i+8)]) for i in [30,38,46]])
          if group[0] == "c":
            x = sym_it(x,C,A[0])
          if group[0] == "d":
            x = sym_it(x,C,A[1],A[0])
          coord_str = "".join(["{:8.3f}".format(a) for a in x])
          new_lines.append(line[:30]+coord_str+line[54:])
      else:
        new_lines.append(line)
    return results, "\n".join(new_lines)

  except:
    return None, pdb_str

def run(command, steps, num_designs=1, visual="none"):

  def run_command_and_get_pid(command):
    pid_file = '/dev/shm/pid'
    os.system(f'nohup {command} > /dev/null & echo $! > {pid_file}')
    with open(pid_file, 'r') as f:
      pid = int(f.read().strip())
    os.remove(pid_file)
    return pid
  def is_process_running(pid):
    try:
      os.kill(pid, 0)
    except OSError:
      return False
    else:
      return True

  run_output = widgets.Output()
  progress = widgets.FloatProgress(min=0, max=1, description='running', bar_style='info')
  display(widgets.VBox([progress, run_output]))

  # clear previous run
  for n in range(steps):
    if os.path.isfile(f"/dev/shm/{n}.pdb"):
      os.remove(f"/dev/shm/{n}.pdb")

  pid = run_command_and_get_pid(command)
  try:
    fail = False
    for _ in range(num_designs):

      # for each step check if output generated
      for n in range(steps):
        wait = True
        while wait and not fail:
          time.sleep(0.1)
          if os.path.isfile(f"/dev/shm/{n}.pdb"):
            pdb_str = open(f"/dev/shm/{n}.pdb").read()
            if pdb_str[-3:] == "TER":
              wait = False
            elif not is_process_running(pid):
              fail = True
          elif not is_process_running(pid):
            fail = True

        if fail:
          progress.bar_style = 'danger'
          progress.description = "failed"
          break

        else:
          progress.value = (n+1) / steps
          if visual != "none":
            with run_output:
              run_output.clear_output(wait=True)
              if visual == "image":
                xyz, bfact = get_ca(f"/dev/shm/{n}.pdb", get_bfact=True)
                fig = plt.figure()
                fig.set_dpi(100);fig.set_figwidth(6);fig.set_figheight(6)
                ax1 = fig.add_subplot(111);ax1.set_xticks([]);ax1.set_yticks([])
                plot_pseudo_3D(xyz, c=bfact, cmin=0.5, cmax=0.9, ax=ax1)
                plt.show()
              if visual == "interactive":
                view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
                view.addModel(pdb_str,'pdb')
                view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':0.5,'max':0.9}}})
                view.zoomTo()
                view.show()
        if os.path.exists(f"/dev/shm/{n}.pdb"):
          os.remove(f"/dev/shm/{n}.pdb")
      if fail:
        progress.bar_style = 'danger'
        progress.description = "failed"
        break

    while is_process_running(pid):
      time.sleep(0.1)

  except KeyboardInterrupt:
    os.kill(pid, signal.SIGTERM)
    progress.bar_style = 'danger'
    progress.description = "stopped"

def run_diffusion(contigs, path, pdb=None, iterations=50,
                  symmetry="none", order=1, hotspot=None,
                  chains=None, add_potential=False,
                  num_designs=1, visual="none", name="test"):

  full_path = f"{path}"
  os.makedirs(full_path, exist_ok=True)
  opts = [f"inference.output_prefix={full_path}",
          f"inference.num_designs={num_designs}"]

  if chains == "": chains = None

  # determine symmetry type
  if symmetry in ["auto","cyclic","dihedral"]:
    if symmetry == "auto":
      sym, copies = None, 1
    else:
      sym, copies = {"cyclic":(f"c{order}",order),
                     "dihedral":(f"d{order}",order*2)}[symmetry]
  else:
    symmetry = None
    sym, copies = None, 1

  # determine mode
  contigs = contigs.replace(","," ").replace(":"," ").split()
  is_fixed, is_free = False, False
  fixed_chains = []
  for contig in contigs:
    for x in contig.split("/"):
      a = x.split("-")[0]
      if a[0].isalpha():
        is_fixed = True
        if a[0] not in fixed_chains:
          fixed_chains.append(a[0])
      if a.isnumeric():
        is_free = True
  if len(contigs) == 0 or not is_free:
    mode = "partial"
  elif is_fixed:
    mode = "fixed"
  else:
    mode = "free"

  # fix input contigs
  if mode in ["partial","fixed"]:
    pdb_str = pdb_to_string(get_pdb(pdb), chains=chains)
    if symmetry == "auto":
      a, pdb_str = run_ananas(pdb_str, path)
      if a is None:
        print(f'ERROR: no symmetry detected')
        symmetry = None
        sym, copies = None, 1
      else:
        if a["group"][0] == "c":
          symmetry = "cyclic"
          sym, copies = a["group"], int(a["group"][1:])
        elif a["group"][0] == "d":
          symmetry = "dihedral"
          sym, copies = a["group"], 2 * int(a["group"][1:])
        else:
          print(f'ERROR: the detected symmetry ({a["group"]}) not currently supported')
          symmetry = None
          sym, copies = None, 1

    elif mode == "fixed":
      pdb_str = pdb_to_string(pdb_str, chains=fixed_chains)

    pdb_filename = f"{full_path}/input.pdb"
    with open(pdb_filename, "w") as handle:
      handle.write(pdb_str)

    parsed_pdb = parse_pdb(pdb_filename)
    opts.append(f"inference.input_pdb={pdb_filename}")
    if mode in ["partial"]:
      iterations = int(80 * (iterations / 200))
      opts.append(f"diffuser.partial_T={iterations}")
      contigs = fix_partial_contigs(contigs, parsed_pdb)
    else:
      opts.append(f"diffuser.T={iterations}")
      contigs = fix_contigs(contigs, parsed_pdb)
  else:
    opts.append(f"diffuser.T={iterations}")
    parsed_pdb = None
    contigs = fix_contigs(contigs, parsed_pdb)

  if hotspot is not None and hotspot != "":
    opts.append(f"ppi.hotspot_res=[{hotspot}]")

  # setup symmetry
  if sym is not None:
    sym_opts = ["--config-name symmetry", f"inference.symmetry={sym}"]
    if add_potential:
      sym_opts += ["'potentials.guiding_potentials=[\"type:olig_contacts,weight_intra:1,weight_inter:0.1\"]'",
                   "potentials.olig_intra_all=True","potentials.olig_inter_all=True",
                   "potentials.guide_scale=2","potentials.guide_decay=quadratic"]
    opts = sym_opts + opts
    contigs = sum([contigs] * copies,[])

  opts.append(f"'contigmap.contigs=[{' '.join(contigs)}]'")
  opts += ["inference.dump_pdb=True","inference.dump_pdb_path='/dev/shm'"]

  print("mode:", mode)
  print("output:", full_path)
  print("contigs:", contigs)

  opts_str = " ".join(opts)
  cmd = f"./RFdiffusion/run_inference.py {opts_str}"
  print(cmd)

  # RUN
  run(cmd, iterations, num_designs, visual=visual)

  # fix pdbs
  cut_of_path = path[:-len(name)]

  for n in range(num_designs):
    pdbs = [f"{cut_of_path}traj/{name}_{n}_pX0_traj.pdb",
            f"{cut_of_path}traj/{name}_{n}_Xt-1_traj.pdb",
            f"{full_path}_{n}.pdb"]
    for pdb in pdbs:
      with open(pdb,"r") as handle: pdb_str = handle.read()
      with open(pdb,"w") as handle: handle.write(fix_pdb(pdb_str, contigs))


  return contigs, copies

installing ColabDesign...
downloading RFdiffusion params...
CPU times: user 5.45 s, sys: 1.06 s, total: 6.5 s
Wall time: 1min 16s


In [ ]:
#@title Get Access to Google Drive or run local { vertical-output: true }
import os
from datetime import datetime
#@markdown - **Attenion** if run locally, if runtime disconnects (highly likely) all data ist lost and all predicitons start from first entry!
#@markdown - **Recommended** run on Google drive to backup the data directly. (Untick local run)
#@markdown - Unfinished RF diffusion runs will still have to be rerun, but partial data can be recovered.
#@markdown - Disabeling local run also allows to use old RFdiff results to just genereate more Sequences with PorteinMPNN"

local_run = False # @param {type:"boolean"}
#@markdown Google drive is default, only activate local if necessary

# Specify the base directory where you want to create the directories
project_directory_name = "Test" # @param {type:"string"}
project_directory_name = project_directory_name.replace(" ", "_")

local_directory = '/content/' + project_directory_name
drive_directory = local_directory

# Add current date to the filename
current_date = datetime.now().strftime('%y%m%d')


local_directory = f'/content/{current_date}' + project_directory_name
drive_directory = local_directory
if 'home_dir_user' not in globals():
    home_dir_user = input("Enter your first name (always enter it the same way!): ")
if not local_run:
  #@title only needed if run on Google cloude computing
  from google.colab import drive
  drive.mount('/content/cloud_drive',force_remount=True)
  drive_directory = f"/content/cloud_drive/MyDrive/{home_dir_user}/RFdiff/{current_date}_" + project_directory_name








Enter your first name (always enter it the same way!): Noah
Mounted at /content/cloud_drive


In [ ]:
%%time
#@title run **RFdiffusion** to generate a backbone
#@markdown  Can be skipped if already successfully run and only more sequences are to be designed with ProteinMPNN.
name = "full_rfdiff_result"
#@markdown Enter which part of the input schould be given into the model (leave empty to consider all input, used for partial diffusion)
#@markdown - For Binder set like this: "B10-196:60", where chain B residue 10-160 defines the target, and we want a 60 aminoacid linker
contigs = "A1-198:65-75" #@param {type:"string"}
#@markdown - Run this cell, upload file will be shown at the bottom. It only accepts manually uploaded files to prevent input mistakes
#@markdown - Double check your input please, especially if the selected residue number are matching to your target residues! Look at the output there the sequence with number is printed again.
pdb = "upload"
iterations = 25 #@param ["25", "50", "100", "150", "200"] {type:"raw"}
#@markdown Set the hotspots like this: "B76,B79,B113" to select residue 76,79,113 of chain B as hotspot
hotspot = "A11,A13" #@param {type:"string"}
hotspot = hotspot.replace(" ", "") #removes spaces from the hotspot input
num_designs = 4 #@param ["1", "2", "4", "8", "16", "32", "64", "128"] {type:"raw"}
visual = "none"
symmetry = "none"
order = 1
chains = ""
add_potential = False


counter = 1

while True:
    # Generate the directory name with a counter
    path = f"{drive_directory}_RFdiff_run_{counter:03d}/{name}"
    # Check if the directory already exists
    if not os.path.exists(path):
        # If it doesn't exist, create the directory and break out of the loop
        os.makedirs(path)
        print(f"Created result directory: {path}")
        break
    counter += 1

flags = {"contigs":contigs,
         "pdb":pdb,
         "order":order,
         "iterations":iterations,
         "symmetry":symmetry,
         "hotspot":hotspot,
         "path":path,
         "chains":chains,
         "add_potential":add_potential,
         "num_designs":num_designs,
         "visual":visual,
         "name":name}

for k,v in flags.items():
  if isinstance(v,str):
    flags[k] = v.replace("'","").replace('"','')

contigs, copies = run_diffusion(**flags)

import pickle

file_path = os.path.join(path, 'contig_variable.pkl')

# Save the variable to the specified directory
with open(file_path, 'wb') as file:
    pickle.dump(contigs, file)

file_path = os.path.join(path, 'copies_variable.pkl')

# Save the variable to the specified directory
with open(file_path, 'wb') as file:
    pickle.dump(copies, file)

file_path = os.path.join(path, 'num_designs_variable.pkl')

# Save the variable to the specified directory
with open(file_path, 'wb') as file:
    pickle.dump(num_designs, file)



In [ ]:
#@title Display 3D structure {run: "auto"}
animate = "none"
color = "plddt"
denoise = True
dpi = 100
from colabdesign.shared.plot import pymol_color_list
from colabdesign.rf.utils import get_ca, get_Ls, make_animation
from string import ascii_uppercase,ascii_lowercase
alphabet_list = list(ascii_uppercase+ascii_lowercase)

if not 'name' in globals():
    print("You are trying to rerun this box without runnning RF diff first, please tick \"restarting_after_shutdown\".")

restarting_after_shutdown = False # @param {type:"boolean"}

if restarting_after_shutdown:
        import pickle
        name = "full_rfdiff_result"

        # Define the drive directory
        home_directory = f"/content/cloud_drive/MyDrive/{home_dir_user}/RFdiff/"



        # Get a list of all directories in the drive directory
        all_dirs = [d for d in os.listdir(home_directory) if os.path.isdir(os.path.join(home_directory, d))]

        # Display a numbered list of directories
        print("Available Directories:")
        for i, d in enumerate(all_dirs, 1):
              print(f"[{i}] {d}")

          # Prompt the user to input a number
        selected_number = input("Enter the number of the directory you want to select: ")

          # Validate user input
        try:
              selected_number = int(selected_number)
              if 1 <= selected_number <= len(all_dirs):
                  selected_dir = all_dirs[selected_number - 1]
                  print(f"Selected directory: {selected_dir}")
              else:
                  print("Invalid number. Please enter a valid number.")
        except ValueError:
              print("Invalid input. Please enter a valid number.")

        file_path_1 = os.path.join(f"{home_directory}{selected_dir}", name, 'contig_variable.pkl')
        file_path_2 = os.path.join(f"{home_directory}{selected_dir}", name, 'copies_variable.pkl')
        file_path_3 = os.path.join(f"{home_directory}{selected_dir}", name, 'num_designs_variable.pkl')

        with open(file_path_1, 'rb') as file:
            contigs = pickle.load(file)
            print(f"Successfully loaded {file_path_1}")

        with open(file_path_2, 'rb') as file:
            copies = pickle.load(file)
            print(f"Successfully loaded {file_path_2}")

        with open(file_path_3, 'rb') as file:
            num_designs = pickle.load(file)
            print(f"Successfully loaded {file_path_3}")


        path = os.path.join(f"{home_directory}{selected_dir}", name)



path_without_last_4 = path[:-len(name)]


def plot_pdb(num=0):
  if denoise:
    pdb_traj = f"{path_without_last_4}traj/{name}_{num}_pX0_traj.pdb"
  else:
    pdb_traj = f"{path_without_last_4}traj/{name}_{num}_Xt-1_traj.pdb"
  if animate in ["none","interactive"]:
    hbondCutoff = 4.0
    view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
    if animate == "interactive":
      pdb_str = open(pdb_traj,'r').read()
      view.addModelsAsFrames(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
    else:
      pdb = f"{path}_{num}.pdb"

      pdb_str = open(pdb,'r').read()
      view.addModel(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
    if color == "rainbow":
      view.setStyle({'cartoon': {'color':'spectrum'}})
    elif color == "chain":
      for n,chain,c in zip(range(len(contigs)),
                              alphabet_list,
                              pymol_color_list):
          view.setStyle({'chain':chain},{'cartoon': {'color':c}})
    else:
      view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':0.5,'max':0.9}}})
    view.zoomTo()
    if animate == "interactive":
      view.animate({'loop': 'backAndForth'})
    view.show()
  else:
    Ls = get_Ls(contigs)
    xyz, bfact = get_ca(pdb_traj, get_bfact=True)
    xyz = xyz.reshape((-1,sum(Ls),3))[::-1]
    bfact = bfact.reshape((-1,sum(Ls)))[::-1]
    if color == "chain":
      display(HTML(make_animation(xyz, Ls=Ls, dpi=dpi, ref=-1)))
    elif color == "rainbow":
      display(HTML(make_animation(xyz, dpi=dpi, ref=-1)))
    else:
      display(HTML(make_animation(xyz, plddt=bfact*100, dpi=dpi, ref=-1)))


if num_designs > 1:
  output = widgets.Output()
  def on_change(change):
    if change['name'] == 'value':
      with output:
        output.clear_output(wait=True)
        plot_pdb(change['new'])
  dropdown = widgets.Dropdown(
      options=[(f'{k}',k) for k in range(num_designs)],
      value=0, description='design:',
  )
  dropdown.observe(on_change)
  display(widgets.VBox([dropdown, output]))
  with output:
    plot_pdb(dropdown.value)
else:
  plot_pdb()



ModuleNotFoundError: No module named 'colabdesign'

In [ ]:
#@title Hacking the script to only check wanted structures {run: "auto"}

#@markdown #*ATTENTION* This script asks for your input
#@markdown Please enter the id of the strucutres that look decent like this: "1,3,4"

# Define the paths for the original and modified files
original_file_path = "colabdesign/rf/designability_test.py"
modified_file_path = "colabdesign/rf/designability_test_modified.py"

# Define the new line to replace the old one
new_line = "  for m in m_values:\n"  # Note the tab character before 'for'

# Read the contents of the original file
with open(original_file_path, "r") as original_file:
    original_content = original_file.readlines()

# Modify the line containing the for loop
modified_content = []
for line in original_content:
    if "for m in range(o.num_designs):" in line:  # Account for the tab character
        modified_content.append(new_line)
    else:
        modified_content.append(line)

# Write the modified contents to the new file
with open(modified_file_path, "w") as modified_file:
    modified_file.writelines(modified_content)

# Define the paths for the original and modified files
original_file_path = "colabdesign/rf/designability_test_modified.py"
modified_file_path = "colabdesign/rf/designability_test_modified.py"

# Define the line to be replaced and the new lines
line_to_replace = '  ag.add(["mpnn_sampling_temp=" ], 0.1,  float, ["sampling temperature used by proteinMPNN"])\n'
new_lines = [
    "  ag.add([\"mpnn_sampling_temp=\" ], 0.1,  float, [\"sampling temperature used by proteinMPNN\"])\n",
    "  ag.add([\"m_str=\" ], None,  str, [\"samples\"])\n",
    "  ag.txt(\"-------------------------------------------------------------------------------------\")\n",
    "  o = ag.parse(argv)\n",
    "  import sys\n",
    "  # Retrieve the values of 'm' from command-line arguments\n",
    "  m_values = list(map(int, o.m_str.split('_')))\n",
    "  # Now you can use 'm_values' in your script\n",
    "  print(\"Values of 'm' received:\", m_values, o.m_str)\n"
]

# Read the contents of the original file
with open(original_file_path, "r") as original_file:
    original_content = original_file.readlines()

# Modify the line containing the specified text
modified_content = []
for line in original_content:
    if line == line_to_replace:  # Check if the line matches the one to be replaced
        modified_content.extend(new_lines)  # Append the new lines
    else:
        modified_content.append(line)

# Write the modified contents to the new file
with open(modified_file_path, "w") as modified_file:
    modified_file.writelines(modified_content)

print(f"Modified file created: {modified_file_path}")

# Prompt the user to enter values for mx separated by commas
#design_choice_input = input("Enter values for mx separated by commas: ")

design_choice_input = "0,3" # @param {type:"string"}

# Convert the input string into a list of integers
mx = [int(x) for x in design_choice_input.split(',')]

# Display the entered values
print("List of designs to feed into ProteinMPNN and AlphaFold2:", mx)

Modified file created: colabdesign/rf/designability_test_modified.py
List of designs to feed into ProteinMPNN and AlphaFold2: [0, 3]


In [ ]:
# @title run **ProteinMPNN** to generate a sequence and **AlphaFold** to validate { display-mode: "form" }
%%time
#@markdown - If more ProteinMPNN results are to be generated with a rerun after a shutdown, check the box
#@markdown - Reasons to also check the box: Change RFdiff input directory
#@markdown - Leave box unchecked if you are just rerunning ProteinMPNN to get more sequneces during a normal campaign.


restarting_after_shutdown = False # @param {type:"boolean"}


if restarting_after_shutdown:
  import pickle
  name = "full_rfdiff_result"

  # Define the drive directory
  home_directory = f"/content/cloud_drive/MyDrive/{home_dir_user}/RFdiff/"



  # Get a list of all directories in the drive directory
  all_dirs = [d for d in os.listdir(home_directory) if os.path.isdir(os.path.join(home_directory, d))]

  # Display a numbered list of directories
  print("Available Directories:")
  for i, d in enumerate(all_dirs, 1):
        print(f"[{i}] {d}")

    # Prompt the user to input a number
  if not 'selected_number' in globals():
    selected_number = input("Enter the number of the directory you want to select: ")
  try:
    if  'selected_number' in globals():
      print(f"Taking the same folder as input that was choose in *Display 3D structure* cell: {path}")
  except ValueError:
        print("Invalid input.")

    # Validate user input
  try:
        selected_number = int(selected_number)
        if 1 <= selected_number <= len(all_dirs):
            selected_dir = all_dirs[selected_number - 1]
            print(f"Selected directory: {selected_dir}")
        else:
            print("Invalid number. Please enter a valid number.")
  except ValueError:
        print("Invalid input. Please enter a valid number.")

  file_path_1 = os.path.join(f"{home_directory}{selected_dir}", name, 'contig_variable.pkl')
  file_path_2 = os.path.join(f"{home_directory}{selected_dir}", name, 'copies_variable.pkl')
  file_path_3 = os.path.join(f"{home_directory}{selected_dir}", name, 'num_designs_variable.pkl')

  with open(file_path_1, 'rb') as file:
      contigs = pickle.load(file)
      print(f"Successfully loaded {file_path_1}")

  with open(file_path_2, 'rb') as file:
      copies = pickle.load(file)
      print(f"Successfully loaded {file_path_2}")

  with open(file_path_3, 'rb') as file:
      num_designs = pickle.load(file)
      print(f"Successfully loaded {file_path_3}")

  path = os.path.join(f"{home_directory}{selected_dir}", name)

if not 'copies' in globals():
    print("Please tick the restarting_after_shutdown box")
    sys.exit(1)

#contigs2 = contigs.replace(" ", ":")
#print(contigs2)
#contigs2 = [f'{contigs2}']


#@markdown - I recommend 16
num_seqs = 16 #@param ["1", "2", "4", "8", "16", "32", "64"] {type:"raw"}
initial_guess = False

#@markdown - Leave at 0, just adjust if necessary and you now why
num_recycles = 0 #@param ["0", "1", "2", "3", "6", "12"] {type:"raw"}
use_multimer = True
#@markdown You can exclude certain aminoacids, like Cystein from being used by ProteinMPNN (Disulfidebridge forming)
rm_aa = "C" #@param {type:"string"}
#@markdown Higher temperature means higher sequence diversity (0.1 for 8 seq is recommended)
mpnn_sampling_temp = 0.1 #@param ["0.0001", "0.1", "0.15", "0.2", "0.25", "0.3", "0.5", "1.0"] {type:"raw"}

if not os.path.isfile("params/done.txt"):
  print("downloading AlphaFold params...")
  while not os.path.isfile("params/done.txt"):
    time.sleep(5)

cut_of_path = path[:-len(name)]

counter = 1

while True:
    # Generate the directory name with a counter
    save_proteinMpnn_results = f"{cut_of_path}results_proteinMPNN_run_{counter:03d}"
    # Check if the directory already exists
    if not os.path.exists(save_proteinMpnn_results):
        # If it doesn't exist, create the directory and break out of the loop
        os.makedirs(save_proteinMpnn_results)
        print(f"Created result directory: {save_proteinMpnn_results}")
        break
    counter += 1

# Convert the list 'mx' to a string
m_str = '_'.join(map(str, mx))

contigs_str = ":".join(contigs)
opts = [f"--pdb={path}_0.pdb",
        f"--loc={save_proteinMpnn_results}",
        f"--contig={contigs_str}",
        f"--copies={copies}",
        f"--num_seqs={num_seqs}",
        f"--num_recycles={num_recycles}",
        f"--rm_aa={rm_aa}",
        f"--mpnn_sampling_temp={mpnn_sampling_temp}",
        f"--num_designs={num_designs}",
        f"--m_str={m_str}"]

if initial_guess: opts.append("--initial_guess")
if use_multimer: opts.append("--use_multimer")
opts = ' '.join(opts)
!python colabdesign/rf/designability_test_modified.py {opts}


delete_their_ranking = f'{save_proteinMpnn_results}/best*'
!rm -f $delete_their_ranking

In [ ]:
# @title Filter and Sort results { display-mode: "form" }

#@markdown Best new binders (if there are any) are found in the "best_binders_redesignes" directory. If ProteinMPNN run multiple time (recommended to generate more condidates) the best binders are collected in the "All_top_binders_redesigns" directory. In both directories you will also find a csv called "info_table_***.csv" with the relevant scores (rounded) and sequences of the best binders.

result_csv_path = os.path.join(save_proteinMpnn_results,"mpnn_results.csv")

print(f"Full result file can be found at: {result_csv_path}")

import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(result_csv_path)
if 1 == 1:
  df[['target','binder']] = df['seq'].str.split('/', expand=True)

  # Drop the original "seq" column if needed
  df = df.drop('seq', axis=1)

  df = df.sort_values(by='i_pae')


  #@markdown ### Set Thresholds
  #@markdown - `plddt`: Pre-set threshold **>= 0.85** *How secure is AF2 that the binder sequence folds like this? Higher = more secure, > 0.9 perfect, 0.8 - 0.7 medium, < 0.6 highly insecure*
  #@markdown - `i_pae`: Pre-set threshold **<= 11** *Interaction score, how sure is AF2 that binder:target interact like this?*
  #@markdown - `rmsd`: Pre-set threshold **<= 10** *Difference between RFdiff binder model and ProteinMPNN+AF2 binder, the higher the more different it is from the binder structure (high rmsd + very low i_pae could be interesting (write me->Noah)*
  #@markdown - `i_ptm`: Pre-set threshold **>= 0.45** *Interaction score, correlates strong with i_pae but is more restrictive*
  #@markdown ***
  #@markdown ### Explorer mode options
  #@markdown - Explore mode should be turned on if the preset setting has not revealed a binder and you want to relax the thesholds **(Not recommended, only use if you know the consequences)**
  Explore_mode = False #@param {type:"boolean"}
  #@markdown - If values are changed without Explorer mode "ON", this will NOT affect the analysis
  plddt_threshold = 0.85
  i_pae_threshold = 11
  rmsd_threshold = 10
  i_ptm_threshold = 0.45

  if Explore_mode:
    # Set threshold values
    plddt_threshold = 0.2 #@param {type:"number"}
    i_pae_threshold = 27 #@param {type:"number"}
    rmsd_threshold = 70 #@param {type:"number"}
    i_ptm_threshold = 0.03 #@param {type:"number"}

  #@markdown ***
  # Assuming df is your DataFrame
  filtered_df = df[
      (df['plddt'] > plddt_threshold) &
      (df['i_pae'] < i_pae_threshold) &
      (df['rmsd'] < rmsd_threshold) &
      (df['i_ptm'] > i_ptm_threshold)
  ].copy()


  columns_to_round = ['mpnn', 'plddt', 'i_ptm', 'i_pae', 'rmsd']

  # Iterate over the specified columns and round the values to two decimal places
  for column in columns_to_round:
      filtered_df.loc[:, column] = filtered_df[column].round(2)
  # Add run info to frame
  name_run = save_proteinMpnn_results[-19:]

  filtered_df['Unnamed: 0'] = filtered_df['Unnamed: 0'].replace(to_replace=df['Unnamed: 0'].unique(), value=name_run)
  # Assuming df is your DataFrame
  filtered_df.rename(columns={'Unnamed: 0': 'run_name'}, inplace=True)

  #Best pdb target directories
  destination_dir = f'{save_proteinMpnn_results}/best_binder_redesigns'
  destination_dir_allstars = f'{cut_of_path}/All_top_binder_redesigns'


  os.makedirs(destination_dir, exist_ok=True)
  cut_of_path = path[:-len(name)]
  os.makedirs(destination_dir_allstars, exist_ok=True)

  import pandas as pd
  import shutil


  if filtered_df.empty:
    sys.exit("No good binder was found! Please rerun ProteinMPNN with more sequences, optionally set recycle = 1 or if all this doesnt help, rerun the RFdiff step with num_designs = 16. Worst case, your input just does not work well with partial diff.")

  # Iterate over the DataFrame rows
  for index, row in filtered_df.iterrows():
      design = row['design']
      n = row['n']
      mpnn = row['mpnn']
      plddt = row['plddt']
      i_ptm = row['i_ptm']
      i_pae = row['i_pae']
      rmsd = row['rmsd']
      target = row['target']
      binder = row['binder']

      # Assuming the file name is a combination of design, n, and target
      source_file_path = f'{save_proteinMpnn_results}/all_pdb/design{design}_n{n}.pdb'

      name_run = save_proteinMpnn_results[-19:]

      # Construct the full paths for destination

      destination_file_name = f'{name_run}_design{design}_n{n}_i-pae_{i_pae}_rmsd_{rmsd}.pdb'

      destination_path_name = os.path.join(destination_dir, destination_file_name)

      # Use shutil.copy to copy the file
      shutil.copy(source_file_path, destination_path_name)


      ## Collecting all the best binders in a single directory
      destination_path_name_allstars = os.path.join(destination_dir_allstars, destination_file_name)
      # Use shutil.copy to copy the file to the allstars folder
      shutil.copy(source_file_path, destination_path_name_allstars)

  # Specify the path where you want to save the CSV file
  destination_csv_name = f'info_table_best_binder_redesigns_{name_run}.csv'

  destination_path_name_csv = os.path.join(destination_dir, destination_csv_name)

  # Write the DataFrame to a CSV file
  filtered_df.to_csv(destination_path_name_csv, index=False)

  # Specify the path where you want to save the CSV file
  destination_csv_name_allstar = f'info_table_best_binder_redesigns_allstars.csv'
  destination_path_name_csv_allstar = os.path.join(destination_dir_allstars, destination_csv_name_allstar)

  if os.path.exists(destination_path_name_csv_allstar):
      # Append the DataFrame to the existing CSV file
      filtered_df.to_csv(destination_path_name_csv_allstar, mode='a', header=False, index=False)
  else:
      # Create the DataFrame to the existing CSV file
      filtered_df.to_csv(destination_path_name_csv_allstar, header=True, index=False)

  # Read the CSV file into a pandas DataFrame
  df_allstars = pd.read_csv(destination_path_name_csv_allstar)

  # Remove duplicate rows based on all columns
  df_allstars = df_allstars.drop_duplicates()

  # Overwrite the original CSV file with the cleaned DataFrame
  df_allstars.to_csv(destination_path_name_csv_allstar, header=True, index=False)

  # Display the filtered DataFrame
  print("These are the best hits in this run fulfilling all minimal criteria of a good binder:target pair. Metrics are rounded")
  print(filtered_df.iloc[:, :8].sort_values(by='i_pae'))

  print('##################################################################################')


  print("These are the best hits in ALL runs fulfilling all minimal criteria of a good binder:target pair. Metrics are rounded")
  df_allstars.sort_values(by='i_pae')




In [ ]:
# @title Display best result and save notebook copy{ display-mode: "form" }
# @markdown MUST run this cell, saves copy of the noteboook to your result folder { display-mode: "form" }
#@markdown - **Highly Recommended** for Reproducibility

#@markdown Personal flavour edition result presentation
import os
import py3Dmol
import ipywidgets as widgets

def plot_pdb(pdb):
    print(pdb)
    hbondCutoff = 4.0
    cut_of_path = path[:-len(name)]
    numer_of_rfdiff_design = pdb[26:27]

    view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
    pdb_str = open(f"{best_binder_path}/{pdb}", 'r').read()
    view.addModel(pdb_str, 'pdb', {'hbondCutoff': hbondCutoff})
    pdb_str_2 = open(f"{cut_of_path}{name}_{numer_of_rfdiff_design}.pdb",'r').read()
    view.addModel(pdb_str_2,'pdb',{'hbondCutoff':hbondCutoff})

    view.setStyle({"model":1},{'cartoon':{}}) #: {'colorscheme': {'prop':'b','gradient': 'roygb','min':0,'max':100}}})
    view.setStyle({"model":0},{'cartoon':{'colorscheme': {'prop':'b','gradient': 'roygb','min':0,'max':100}}})
    view.zoomTo()
    view.show()
try:
  best_binder_path = destination_dir
  pdb_files = [f for f in os.listdir(best_binder_path) if f.endswith('.pdb')]

  if len(pdb_files) > 1:
      dropdown = widgets.Dropdown(
          options=pdb_files,
          value=pdb_files[0],
          description='Select a pdb file:',
      )
      output = widgets.Output()
      display(widgets.VBox([dropdown, output]))

      with output:
          plot_pdb(dropdown.value)

      def on_change(change):
          if change['name'] == 'value':
              with output:
                  output.clear_output(wait=True)
                  plot_pdb(change['new'])

      dropdown.observe(on_change)
  else:
      plot_pdb(pdb_files[0])
except:
  print("Display of normal RFdiff results is not supported yet, only 3DDiff Binder design")

if not local_run:
  import shutil

  # Define the source and destination paths
  source_path = '/content/cloud_drive/MyDrive/Colab Notebooks/Selective_RFdiff_with_Backup.ipynb'
  destination_directory = save_proteinMpnn_results  # Assuming jobname is the variable containing the directory path

  # Specify the destination path
  destination_path = os.path.join(destination_directory, f'Selective_RFdiff_with_Backup_for_{project_directory_name}.ipynb')

  # Copy the file
  shutil.copy(source_path, destination_path)
  print(f"Sucessfullly saved a copy at {destination_path}")



**Instructions**
---
---

Use `contigs` to define continious chains. Use a `:` to define multiple contigs and a `/` to define mutliple segments within a contig.
For example:


**binder design**
- `contigs='A:50'` `pdb='4N5T'` - diffuse a **binder** of length 50 to chain A of defined PDB.
- `contigs='E6-155:70-100'` `pdb='5KQV'` `hotspot='E64,E88,E96'` - diffuse a **binder** of length 70 to 100 (sampled randomly) to chain E and defined hotspot(s).

*hints and tips*
- `pdb=''` leave blank to get an upload prompt
- `contigs='50-100'` use dash to specify a range of lengths to sample from